In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.8/112.6 GB disk)


In [ ]:
!mkdir {HOME}/datasets
%cd {HsetsOME}/data

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("ntlh1805")
project = rf.workspace("stubbornstrawsberries").project("fisheye8k")
version = project.version(6)
dataset = version.download("yolo11")

/content/datasets
loading Roboflow workspace...
loading Roboflow workspace...
loading Roboflow project...

Version export complete for yolo11 format



Extracting Dataset Version Zip to Fisheye8K-6 in yolo11:: 100%|██████████| 12214/12214 [00:01<00:00, 7007.08it/s]


In [ ]:
import json

with open('/content/datasets/Fisheye8K-6/train/_annotations.coco.json') as f:
    coco = json.load(f)
    classes = [cat['name'] for cat in coco['categories']]
    for i, name in enumerate(classes):
        print(f"  {i}: {name}")


  0: bikes-cars-pedestrians-bus-truck
  1: 0
  2: 1
  3: 2
  4: 3
  5: 4


In [ ]:
yaml_content = """
path: /content/Fisheye8K-6
train: train
val: valid
test: test

names:
   0: bikes-cars-pedestrians-bus-truck
   1: 0
  2: 1
  3: 2
  4: 3
  5: 4
"""

with open("/content/coco_fisheye.yaml", "w") as f:
    f.write(yaml_content)


In [ ]:
!yolo task=detect mode=train model=yolo11m.pt data={dataset.location}/data.yaml epochs=10 imgsz=640 plots=True


100% 38.8M/38.8M [00:00<00:00, 150MB/s]
New https://pypi.org/project/ultralytics/8.3.152 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/datasets/Fisheye8K-6/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/datasets/runs /content/drive/MyDrive/


In [ ]:
import cv2
import numpy as np

In [ ]:
model = YOLO("/content/datasets/runs/detect/train/weights/best.pt")
input_folder = "/content/datasets/Fisheye8K-6/test/images"  # thư mục ảnh gốc
crop_dir = "/content/preprocessed3"
os.makedirs(crop_dir, exist_ok=True)

for fname in os.listdir(input_folder):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png')): continue
    img_path = os.path.join(input_folder, fname)
    image = cv2.imread(img_path)
    if image is None: continue

    results = model(img_path)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        class_id = int(classes[i])
        crop = image[y1:y2, x1:x2]
        if crop.size == 0: continue

        out_dir = os.path.join(crop_dir, f"class_{class_id}")
        os.makedirs(out_dir, exist_ok=True)
        fname_out = f"{os.path.splitext(fname)[0]}_{class_id}_{datetime.now().strftime('%H%M%S%f')}.jpg"
        cv2.imwrite(os.path.join(out_dir, fname_out), crop)


image 1/1 /content/datasets/Fisheye8K-6/test/images/camera1_A_319_png.rf.48a89fc89625735450b9de1d6ec1af4c.jpg: 640x640 1 0, 35 1s, 15 2s, 4 3s, 36.9ms
Speed: 2.4ms preprocess, 36.9ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/datasets/Fisheye8K-6/test/images/camera4_M_461_png.rf.1db310109f487483fa1df30904f25b00.jpg: 640x640 4 1s, 3 2s, 1 4, 36.9ms
Speed: 1.9ms preprocess, 36.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/datasets/Fisheye8K-6/test/images/camera7_A_159_png.rf.579fccfa5079cd67e768d905d71de515.jpg: 640x640 1 0, 6 1s, 1 2, 36.9ms
Speed: 2.0ms preprocess, 36.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/datasets/Fisheye8K-6/test/images/camera4_E_1_png.rf.e20b9b6f932c7ffc183e947f51b376c2.jpg: 640x640 7 1s, 9 2s, 1 4, 36.9ms
Speed: 2.1ms preprocess, 36.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/datasets/Fi

In [ ]:
# ==============================================================================
# GIAI ĐOẠN 2: ĐỊNH NGHĨA CÔNG CỤ TIỀN XỬ LÝ
# ==============================================================================
# In thông báo bắt đầu Giai đoạn 2
print("🚀 Giai đoạn 2: Định nghĩa công cụ tiền xử lý...")

# Hàm kiểm tra tính hợp lệ của ảnh
def check_image_validity(img, img_path=None):
    # Bắt đầu kiểm tra lỗi
    try:
        # Nếu img là None và có img_path, đọc ảnh từ đường dẫn
        if img is None and img_path:
            # Kiểm tra file có tồn tại không
            if not os.path.exists(img_path):
                return False, f"Ảnh không tồn tại: {img_path}"
            # Đọc ảnh bằng OpenCV
            img = cv2.imread(img_path)
            # Kiểm tra nếu không đọc được ảnh
            if img is None:
                return False, f"Không đọc được: {img_path}"
        # Kiểm tra nếu img vẫn là None
        if img is None:
            return False, "Ảnh là None."
        # Kiểm tra định dạng ảnh (2D hoặc 3D)
        if len(img.shape) not in [2, 3]:
            return False, f"Định dạng không hợp lệ: shape {img.shape}"
        # Kiểm tra số kênh màu nếu là ảnh màu
        if len(img.shape) == 3 and img.shape[2] not in [1, 3, 4]:
            return False, f"Kênh màu không hợp lệ: {img.shape[2]}"
        # Kiểm tra nếu ảnh rỗng
        if img.size == 0:
            return False, "Ảnh rỗng"
        # Trả về kết quả hợp lệ
        return True, "Ảnh hợp lệ."
    # Xử lý lỗi nếu có
    except Exception as e:
        return False, f"Lỗi kiểm tra: {str(e)}"

# Hàm phân tích đặc tính ảnh
def analyze_image_properties(img):
    # Bắt đầu phân tích
    try:
        # Chuyển ảnh sang grayscale nếu là ảnh màu
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
        # Tính độ sáng trung bình
        mean_brightness = np.mean(gray)
        # Kiểm tra ảnh quá tối
        is_too_dark = mean_brightness < 50
        # Kiểm tra ảnh quá sáng
        is_too_bright = mean_brightness > 200
        # Tính độ mờ bằng Laplacian
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        # Kiểm tra ảnh mờ
        is_blurry = laplacian_var < 100
        # Chuyển ảnh sang không gian màu LAB
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB) if len(img.shape) == 3 else cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_GRAY2BGR), cv2.COLOR_BGR2LAB)
        # Tách kênh L (độ sáng)
        l, _, _ = cv2.split(lab)
        # Tính độ tương phản
        contrast = np.std(l)
        # Kiểm tra tương phản thấp
        is_low_contrast = contrast < 30
        # Mặc định không có bão hòa thấp
        is_low_saturation = False
        # Nếu là ảnh màu, kiểm tra độ bão hòa
        if len(img.shape) == 3 and img.shape[2] == 3:
            # Chuyển sang không gian màu HSV
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            # Tính độ bão hòa trung bình
            saturation = hsv[:, :, 1].mean()
            # Kiểm tra bão hòa thấp
            is_low_saturation = saturation < 50
        # Tạo dictionary chứa gợi ý xử lý
        suggestions = {
            'apply_contrast': is_too_dark or is_too_bright or is_low_contrast,
            'apply_sharpen': is_blurry,
            'apply_augmentation': is_low_saturation or is_low_contrast,
            'issues': []
        }
        # Thêm vấn đề nếu ảnh quá tối
        if is_too_dark:
            suggestions['issues'].append(f"Quá tối (sáng: {mean_brightness:.1f})")
        # Thêm vấn đề nếu ảnh quá sáng
        if is_too_bright:
            suggestions['issues'].append(f"Quá sáng (sáng: {mean_brightness:.1f})")
        # Thêm vấn đề nếu ảnh mờ
        if is_blurry:
            suggestions['issues'].append(f"Mờ (Laplacian: {laplacian_var:.1f})")
        # Thêm vấn đề nếu tương phản thấp
        if is_low_contrast:
            suggestions['issues'].append(f"Tương phản thấp (std: {contrast:.1f})")
        # Thêm vấn đề nếu bão hòa thấp
        if is_low_saturation:
            suggestions['issues'].append(f"Bão hòa thấp (sat: {saturation:.1f})")
        # Trả về gợi ý
        return suggestions
    # Xử lý lỗi nếu có
    except Exception as e:
        return {'issues': [f"Lỗi phân tích: {str(e)}"], 'apply_contrast': False, 'apply_sharpen': False, 'apply_augmentation': False}

# Hàm làm sắc nét ảnh
def sharpen_image(img):
    # Tạo kernel cho bộ lọc làm sắc nét
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    # Áp dụng bộ lọc
    return cv2.filter2D(img, -1, kernel)

# Hàm tải tham số hiệu chỉnh Fisheye
def load_fisheye_params():
    # Định nghĩa ma trận camera K
    K = np.array([[1000, 0, 960], [0, 1000, 540], [0, 0, 1]], dtype=np.float32)
    # Định nghĩa hệ số méo D
    D = np.array([0.1, 0.01, 0.001, 0.0001], dtype=np.float32)
    # Trả về K và D
    return K, D

# Hàm letterbox để resize ảnh mà giữ tỷ lệ
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114)):
    # Lấy kích thước gốc của ảnh
    shape = img.shape[:2]
    # Tính tỷ lệ resize
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    # Tính kích thước mới sau resize
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    # Tính padding cần thêm
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
    # Chia đều padding cho hai bên
    dw /= 2
    dh /= 2
    # Resize ảnh nếu cần
    if shape[::-1] != new_unpad:
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    # Tính vị trí padding
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    # Thêm padding để tạo letterbox
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

# Hàm tăng cường độ tương phản
def enhance_contrast(img):
    # Chuyển ảnh sang không gian màu LAB
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    # Tách các kênh L, A, B
    l, a, b = cv2.split(lab)
    # Tạo bộ lọc CLAHE để tăng tương phản
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    # Áp dụng CLAHE lên kênh L
    cl = clahe.apply(l)
    # Gộp lại các kênh
    limg = cv2.merge((cl, a, b))
    # Chuyển lại sang BGR
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

# Hàm khử méo Fisheye
def undistort_fisheye(img, K, D):
    # Lấy kích thước ảnh
    h, w = img.shape[:2]
    # Tính ma trận camera mới
    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, (w, h), np.eye(3))
    # Tạo map để khử méo
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), new_K, (w, h), cv2.CV_16SC2)
    # Áp dụng map để khử méo
    return cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

# Hàm tăng cường dữ liệu ảnh (photometric)
def augment_image_photometric(img):
    # Định nghĩa pipeline tăng cường với RandomBrightnessContrast và RandomGamma
    transform = A.Compose([A.RandomBrightnessContrast(p=0.5), A.RandomGamma(p=0.5)])
    # Áp dụng pipeline lên ảnh
    return transform(image=img)['image']

# Hàm pipeline tiền xử lý ảnh tổng hợp
def ultimate_preprocess_pipeline(image_path=None, img=None, is_fisheye=None, K_fisheye=None, D_fisheye=None, fix_contrast=False, apply_augmentation=False, apply_sharpen=False):
    # Kiểm tra tính hợp lệ của ảnh
    is_valid, error_msg = check_image_validity(img, image_path)
    # Nếu không hợp lệ, in lỗi và trả về None
    if not is_valid:
        print(f"❌ Lỗi xử lý: {error_msg}")
        return None
    # Nếu img là None, đọc ảnh từ đường dẫn
    if img is None:
        # Đọc ảnh từ file
        img = cv2.imread(image_path)
    # Sao chép ảnh để xử lý
    processed_img = img.copy()
    # Chuyển ảnh grayscale sang BGR nếu cần
    if len(processed_img.shape) == 2:
        processed_img = cv2.cvtColor(processed_img, cv2.COLOR_GRAY2BGR)
    # Chuyển ảnh BGRA sang BGR nếu có kênh alpha
    elif processed_img.shape[2] == 4:
        processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGRA2BGR)
    # Phân tích đặc tính ảnh
    properties = analyze_image_properties(processed_img)
    # In các vấn đề nếu có
    if properties['issues']:
        print(f"⚠️ Phân tích: {', '.join(properties['issues'])}")
    # Quyết định áp dụng các bước xử lý
    final_fix_contrast = fix_contrast or properties['apply_contrast']
    final_apply_augmentation = apply_augmentation or properties['apply_augmentation']
    final_apply_sharpen = apply_sharpen or properties['apply_sharpen']
    # Áp dụng tăng cường dữ liệu nếu cần
    if final_apply_augmentation:
        processed_img = augment_image_photometric(processed_img)
    # Khử méo Fisheye nếu được yêu cầu
    if is_fisheye and K_fisheye is not None and D_fisheye is not None:
        processed_img = undistort_fisheye(processed_img, K_fisheye, D_fisheye)
    # Tăng tương phản nếu cần
    if final_fix_contrast:
        processed_img = enhance_contrast(processed_img)
    # Làm sắc nét nếu cần
    if final_apply_sharpen:
        processed_img = sharpen_image(processed_img)
    # Áp dụng letterbox để resize ảnh
    return letterbox(processed_img, new_shape=(640, 640))

# In thông báo hoàn tất Giai đoạn 2
print("✅ Giai đoạn 2 hoàn tất.")
# In đường phân cách
print("-" * 70)

🚀 Giai đoạn 2: Định nghĩa công cụ tiền xử lý...
✅ Giai đoạn 2 hoàn tất.
----------------------------------------------------------------------


In [ ]:
!cp -r /content/cropped_objects2 /content/drive/MyDrive/

In [ ]:
!cp -r /content/preprocessed3 /content/drive/MyDrive/

In [ ]:
import os
import cv2

# Đường dẫn input/output
input_dir = "/content/preprocessed3"              # ảnh crop từ YOLO (class_x/)
output_dir = "/content/preprocessed_clean_v2"    # nơi lưu ảnh sau xử lý

# Load thông số fisheye
K_fisheye, D_fisheye = load_fisheye_params()

# Tạo thư mục output nếu chưa có
os.makedirs(output_dir, exist_ok=True)

# Duyệt qua từng class
for cls_name in os.listdir(input_dir):
    cls_path = os.path.join(input_dir, cls_name)
    if not os.path.isdir(cls_path): continue

    save_cls_path = os.path.join(output_dir, cls_name)
    os.makedirs(save_cls_path, exist_ok=True)

    for file in os.listdir(cls_path):
        if not file.lower().endswith((".jpg", ".png", ".jpeg")):
            continue

        img_path = os.path.join(cls_path, file)
        save_path = os.path.join(save_cls_path, file)

        processed_img = ultimate_preprocess_pipeline(
            image_path=img_path,
            is_fisheye=True,
            K_fisheye=K_fisheye,
            D_fisheye=D_fisheye,
            fix_contrast=True,
            apply_augmentation=True,
            apply_sharpen=True
        )

        if processed_img is not None:
            cv2.imwrite(save_path, processed_img)
            print(f"✅ Saved: {save_path}")
        else:
            print(f"⚠️ Skipped: {img_path}")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
⚠️ Phân tích: Tương phản thấp (std: 27.1), Bão hòa thấp (sat: 49.5)
✅ Saved: /content/preprocessed_clean_v2/class_4/camera4_M_534_png.rf.e34aa1eee7130d9b99bebbf1fa8e91dc_4_151357031027.jpg
⚠️ Phân tích: Bão hòa thấp (sat: 0.0)
✅ Saved: /content/preprocessed_clean_v2/class_3/camera4_N_95_png.rf.83d4be069a4f9731e5d3596daccfb399_3_151352458055.jpg
⚠️ Phân tích: Bão hòa thấp (sat: 45.1)
✅ Saved: /content/preprocessed_clean_v2/class_3/camera1_A_168_png.rf.97ca73d18c41beb472c5666b4580f1e1_3_151349992164.jpg
⚠️ Phân tích: Tương phản thấp (std: 22.4), Bão hòa thấp (sat: 32.6)
✅ Saved: /content/preprocessed_clean_v2/class_3/camera4_M_60_png.rf.c8789c716116873299fc1c782994b4c4_3_151357517575.jpg
✅ Saved: /content/preprocessed_clean_v2/class_3/camera1_A_385_png.rf.c80be443b8942fb79722d9fd75f0033b_3_151356926504.jpg
⚠️ Phân tích: Bão hòa thấp (sat: 30.6)
✅ Saved: /content/preprocessed_clean_v2/class_3/camera2_A_274_png.rf.5fd3fd0157ec856d10e

In [ ]:
import os
import cv2
import shutil
import numpy as np
import torch
from ultralytics import YOLO
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import albumentations as A
from datetime import datetime

In [ ]:
import os, shutil
import random

source_dir = "/content/preprocessed_clean_v2"
target_base = "/content/cnn_data3"
train_dir = os.path.join(target_base, "train")
val_dir = os.path.join(target_base, "val")

split_ratio = 0.8  # 80% train, 20% val

# Tạo thư mục đích
for d in [train_dir, val_dir]:
    os.makedirs(d, exist_ok=True)

for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    split_point = int(len(images) * split_ratio)
    train_imgs = images[:split_point]
    val_imgs = images[split_point:]

    # Tạo thư mục class trong train và val
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

    # Copy ảnh
    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
    for img in val_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

print("✅ Đã chia xong dataset thành train và val.")
print("📊 Train split:", count_class_samples(train_dir))
print("📊 Val split:", count_class_samples(val_dir))


✅ Đã chia xong dataset thành train và val.
📊 Train split: {'class_2': 1076, 'class_0': 58, 'class_4': 80, 'class_3': 160, 'class_1': 1995}
📊 Val split: {'class_2': 269, 'class_0': 15, 'class_4': 20, 'class_3': 41, 'class_1': 499}


In [ ]:
!cp -r /content/cnn_data3 /content/drive/MyDrive/

In [ ]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

# Thiết lập preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Đường dẫn mới đã chia
train_dir = "/content/cnn_data3/train"
val_dir = "/content/cnn_data3/val"

# Dataloader
train_ds = datasets.ImageFolder(train_dir, transform=transform)
val_ds = datasets.ImageFolder(val_dir, transform=transform)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)


In [ ]:
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 5)  # 5 lớp của bạn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 124MB/s] 


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Epochs
for epoch in range(5):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} ✅ Train Loss: {total_loss / len(train_loader):.4f}")


Epoch 1 ✅ Train Loss: 0.7355
Epoch 2 ✅ Train Loss: 0.4933
Epoch 3 ✅ Train Loss: 0.3799
Epoch 4 ✅ Train Loss: 0.2790
Epoch 5 ✅ Train Loss: 0.2286


In [ ]:
correct, total = 0, 0
model.eval()
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"✅ Accuracy: {correct / total:.2%}")


✅ Accuracy: 87.32%


In [ ]:
torch.save(model.state_dict(), "/content/efficientnet_finetuned.pt")

In [ ]:
!cp -r /content/efficientnet_finetuned.pt /content/drive/MyDrive/